In [1]:
import { z } from "zod"
import zodToJsonSchema from "zod-to-json-schema"
import { ChatOpenAI } from "@langchain/openai"
import { ChatPromptTemplate, PromptTemplate } from "@langchain/core/prompts"
import { RunnableSequence, RunnableBranch } from "@langchain/core/runnables"
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools"
import { StringOutputParser } from "@langchain/core/output_parsers"


In [2]:
import { load } from "dotenv"
const env = await load({
  envPath: ".env.local",
})

const process = { env }

const chatOptions = {
  openAIApiKey: process.env.Tongyi_API_KEY,
  temperature: 0,
  modelName: "qwen-plus",
  configuration: {
    baseURL: process.env.BASE_URL,
  },
}


模仿人类的合作方式，构建多个 llm chain 来相互合作，最终得到一个在多个领域表现都不错的模型。


首先创建一个入口 llm 来对用户的问题进行分类，然后根据问题的分类，路由到不同的专业 llm chain 来回答问题。


In [3]:
const classifySchema = z.object({
  type: z.enum(["科普", "编程", "一般问题"]).describe("用户提问的分类"),
})

const model = new ChatOpenAI({
  ...chatOptions,
  temperature: 0,
})

const modelWithTools = model.bind({
  tools: [
    {
      type: "function",
      function: {
        name: "classifyQuestion",
        description: "对用户的提问进行分类",
        parameters: zodToJsonSchema(classifySchema),
      },
    },
  ],
  tool_choice: {
    type: "function",
    function: {
      name: "classifyQuestion",
    },
  },
})

const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `仔细思考，你有充足的时间进行严谨的思考，然后对用户的问题进行分类，
    当你无法分类到特定分类时，可以分类到 "一般问题"`,
  ],
  ["human", "{input}"],
])

const classifyChain = RunnableSequence.from([
  prompt,
  modelWithTools,
  new JsonOutputToolsParser(),
  (input) => {
    const type = input[0]?.args?.type
    return type ? type : "一般问题"
  },
])


In [4]:
const res = await classifyChain.invoke({
  input: "鲸鱼是哺乳动物么？",
})

console.log(res)


科普


根据不同的问题构建不同的专家 chain，这些专家可以访问不同的数据库来避免数据的污染


In [5]:
const answeringModel = new ChatOpenAI({
  ...chatOptions,
  templature: 0.7,
})

const sciencePrompt = PromptTemplate.fromTemplate(
  `作为一位科普专家，你需要解答以下问题，尽可能提供详细、准确和易于理解的答案：

  问题：{input}
  答案：`
)

const programmingPrompt = PromptTemplate.fromTemplate(
  `作为一位编程专家，你需要解答以下编程相关的问题，尽可能提供详细、准确和实用的答案：

问题：{input}
答案：`
)

const generalPrompt = PromptTemplate.fromTemplate(
  `请回答以下一般性问题，尽可能提供全面和有深度的答案：

问题：{input}
答案：`
)

const scienceChain = RunnableSequence.from([
  sciencePrompt,
  answeringModel,
  new StringOutputParser(),
  {
    output: (input) => input,
    role: () => "科普专家",
  },
])

const programmingChain = RunnableSequence.from([
  programmingPrompt,
  answeringModel,
  new StringOutputParser(),
  {
    output: (input) => input,
    role: () => "编程大师",
  },
])

const generalChain = RunnableSequence.from([
  generalPrompt,
  answeringModel,
  new StringOutputParser(),
  {
    output: (input) => input,
    role: () => "通识专家",
  },
])


路由


In [6]:
const branch = RunnableBranch.from([
  [(input) => input.type.includes("科普"), scienceChain],
  [(input) => input.type.includes("编程"), programmingChain],
  generalChain,
])


In [7]:
const outputTemplate = PromptTemplate.fromTemplate(
  `感谢您的提问，这是来自 {role} 的专业回答：
  
  {output}
  `
)

const finalChain = RunnableSequence.from([
  {
    type: classifyChain,
    input: (input) => input.input,
  },
  branch,
  (input) => outputTemplate.format(input),
])


In [8]:
const res = await finalChain.invoke({
  "input": "鲸鱼是哺乳动物么？"
})

console.log(res)

感谢您的提问，这是来自 科普专家 的专业回答：
  
  **答案：是的，鲸鱼是哺乳动物。**

虽然鲸鱼生活在水中，看起来像鱼类，但实际上它们是哺乳动物。这是基于它们具有哺乳动物的关键特征。以下是详细的解释：

### 1. **呼吸空气**
   鲸鱼是用肺呼吸的，而不是像鱼类那样用鳃呼吸。鲸鱼必须定期浮到水面上通过头顶的“气孔”（即呼吸孔）呼吸空气。它们在水下屏住呼吸，然后浮出水面换气。

### 2. **胎生和哺乳**
   鲸鱼是胎生动物，母鲸会在体内孕育幼崽，而不是像鱼类那样产卵。幼鲸出生后，母鲸会通过乳头分泌乳汁喂养幼崽，这也是“哺乳动物”这个名字的由来。

### 3. **体温恒定**
   鲸鱼是恒温动物，意味着它们的体温不会随外界环境变化而波动。相比之下，鱼类是变温动物，体温会随着水温的变化而变化。

### 4. **毛发和汗腺**
   虽然鲸鱼的毛发非常少，几乎看不见，但它们仍然有少量的毛发，尤其是在幼鲸的头部。此外，鲸鱼体内有汗腺，尽管它们不像陆地哺乳动物那样通过汗腺散热，而是通过血液流动调节体温。

### 5. **骨骼结构**
   鲸鱼的骨骼结构与其他哺乳动物相似。例如，它们的前肢进化成了鳍状肢，但内部骨骼结构与陆地哺乳动物的前肢相似。鲸鱼的祖先实际上是生活在陆地上的哺乳动物，后来逐渐演化为水生动物。

### 总结：
尽管鲸鱼生活在水中，但它们具备所有哺乳动物的基本特征：用肺呼吸、胎生哺乳、体温恒定、有毛发和汗腺等。因此，鲸鱼属于哺乳动物类群，而不是鱼类。

希望这个解释能帮助你更好地理解为什么鲸鱼是哺乳动物！
  
